# Sample training script with naive competitive self-play.

In [1]:
# # GPU Diagnostics
# import torch
# print("="*50)
# print("GPU Diagnostics:")
# print("="*50)
# print(f"PyTorch version: {torch.__version__}")
# print(f"CUDA available: {torch.cuda.is_available()}")
# print(f"CUDA version (built with): {torch.version.cuda}")
# if torch.cuda.is_available():
#     print(f"Number of GPUs: {torch.cuda.device_count()}")
#     print(f"Current GPU: {torch.cuda.current_device()}")
#     print(f"GPU Name: {torch.cuda.get_device_name(0)}")
#     print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
# else:
#     print("❌ No GPU detected by PyTorch!")
#     print("\nPossible solutions:")
#     print("1. Install PyTorch with CUDA support:")
#     print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
#     print("2. Check NVIDIA drivers: nvidia-smi")
#     print("3. Verify CUDA toolkit is installed")
# print("="*50)

### Switch directory in Google drive so as to import CDA env.


In [2]:
is_colab = False
# is_colab = True

# is_1st_run = False
is_1st_run = True

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# %cd "/root/ray_results/"
# !ls -l
# #!rm -rf PPO_continuousDoubleAuction-v0_*
# !ls -l
# !pwd

# %cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
# !ls -l

# #!pip install -r requirements.txt

# #!pip install tensorflow==2.2.0
# #!pip install ray[rllib]==0.8.5

# #!pip show tensorflow
# #!pip show ray

# #!pip freeze > requirements.txt

In [4]:
if is_colab == False and is_1st_run == True:
    !pip install sortedcontainers
    !!pip install scikit-learn
    !pip install tabulate
    !pip install matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
# !pip install -U ipywidgets
# !pip install pettingzoo

In [6]:
# !pwd

In [7]:
# os.chdir('..')

In [8]:
# !pip install -e.

In [9]:
# !pip uninstall continuousDoubleAuction
# !pip uninstall continuousDoubleAuction-v0

In [10]:
# !pip show continuousDoubleAuction
# !pip show continuousDoubleAuction-v0

In [11]:
# os.chdir('gym_continuousDoubleAuction')

In [12]:
# !pwd

In [13]:
if is_colab == True:
    !pip install -U ray[rllib]==2.48.0

    from google.colab import drive
    drive.mount('/content/gdrive')

    %cd 'gdrive/MyDrive/Colab Notebooks/MARL/gym-continuousDoubleAuction'

In [14]:
import ray
import ray.rllib
import ray.tune

print("Ray version:", ray.__version__)

2025-12-21 08:34:27,008	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-12-21 08:34:28,199	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Ray version: 2.48.0


In [15]:
!pip show gymnasium

Name: gymnasium
Version: 1.0.0
Summary: A standard API for reinforcement learning and a diverse set of reference environments (formerly Gym).
Home-page: https://farama.org
Author: 
Author-email: Farama Foundation <contact@farama.org>
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: cloudpickle, farama-notifications, numpy, typing-extensions
Required-by: 


### Imports

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"
os.environ['PYTHONWARNINGS'] = 'ignore::DeprecationWarning'

import argparse

# import gym
import gymnasium as gym

import random
import numpy as np

from collections import defaultdict
from typing import Dict

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.policy import Policy

from ray.rllib.algorithms.ppo import PPOConfig

from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.sample_batch import SampleBatch

from ray.rllib.algorithms.callbacks import DefaultCallbacks

from ray.tune.logger import pretty_print

import sys
if "../" not in sys.path:
    sys.path.append("../")


from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

from gym_continuousDoubleAuction.train.model.model_handler import CustomRLModule

from gym_continuousDoubleAuction.train.policy.policy_handler import (
    # make_RandomPolicy,
    # gen_policy,
    # set_agents_policies,
    # create_train_policy_list,
    create_multi_agent_config,
    policy_mapping_fn,
    # create_and_train_algorithm,
)
from gym_continuousDoubleAuction.train.weight.weight_handler import (
    get_trained_policies_name, get_max_reward_ind, cp_weight)
from gym_continuousDoubleAuction.train.storage.store_handler import storage



from gym_continuousDoubleAuction.train.callbk.callbk_handler import store_eps_hist_data
from gym_continuousDoubleAuction.train.callbk.self_play_callback_mod import SelfPlayCallback



from gym_continuousDoubleAuction.train.logger.log_handler import (
    create_dir, log_g_store, load_g_store)
from gym_continuousDoubleAuction.train.plotter.plot_handler import (
    plot_storage, plot_LOB_subplot, plot_sum_ord_imb, plot_mid_prices)
from gym_continuousDoubleAuction.train.helper.helper import (
    ord_imb, sum_ord_imb, mid_price)


print(f'Imports all OK.')

Imports all OK.


### Global


In [17]:
# CDA_env args
num_agents = 4
num_trained_agent = 2 #
num_policies = num_agents # Each agent is using a separate policy
num_of_traders = num_agents
tape_display_length = 10
tick_size = 1
init_cash = 1000000
# max_step = 4096 # per episode, -1 in arg. (~7.2s/1000steps/iter)
max_step = 1024 # per episode, -1 in arg. (~7.2s/1000steps/iter)
is_render = False

# RLlib config
# train_policy_list = create_train_policy_list(num_trained_agent, "policy_")
#num_cpus = 0.25
num_gpus = 0.75 #0
num_cpus_per_worker = 0.25
num_gpus_per_worker = 0
num_workers = 2
num_envs_per_worker = 4
batch_mode = "complete_episodes"
# rollout_fragment_length = 128
num_episodes_per_iter = 4
# agent_time_step_per_episode = max_step * num_agents
# train_batch_size = agent_time_step_per_episode * num_episodes_per_iter
train_batch_size = max_step * num_episodes_per_iter
# sgd_minibatch_size = 256
num_iters = 16

# log_base_dir = "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/results/"
log_base_dir = "results/"
log_dir = log_base_dir + "ray_results/"

# Chkpt & restore
local_dir = log_base_dir + "chkpt/"
chkpt_freq = 10
chkpt = 320
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = True # True / False

# log & load
log_g_store_dir = log_base_dir + "log_g_store/"
create_dir(log_base_dir)
create_dir(log_g_store_dir)

# Environment configuration
env_config = {
    "num_of_agents": num_agents,
    "init_cash": init_cash,
    "tick_size": tick_size,
    "tape_display_length": tape_display_length,
    "max_step": max_step,
    "is_render": is_render
}

# get obs & act spaces from dummy CDA env
# single_CDA_env = continuousDoubleAuctionEnv(
#     num_of_traders,
#     init_cash,
#     tick_size,
#     tape_display_length,
#     max_step,
#     is_render)
single_CDA_env = continuousDoubleAuctionEnv(env_config)
obs_space = single_CDA_env.get_observation_space(single_CDA_env.agents[0])
act_space = single_CDA_env.get_action_space(single_CDA_env.agents[0])
print(single_CDA_env.agents)  # Should be a non-empty list
print(single_CDA_env.get_observation_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space
print(single_CDA_env.get_action_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space

def env_creator(env_config):
    return continuousDoubleAuctionEnv(env_config)

# Register environment with ray.tune - this is the key fix!
tune.register_env("continuousDoubleAuction-v0", env_creator)

# register custom model (neural network)
ModelCatalog.register_custom_model("model_disc", CustomRLModule)

ray.shutdown()
# start ray
ray.init(
    ignore_reinit_error=True,
    log_to_driver=True,
    num_cpus=2,
    dashboard_host="127.0.0.1",  # replaces webui_host
    dashboard_port=8265,          # default port; replaces webui_port
    # include_dashboard=True,        # default True
    include_dashboard=False,        # default True

)

# Global storage, a ray actor that run on it's own process & it needs to be declared after ray.init().
g_store = storage.options(name="g_store", lifetime="detached").remote(num_agents)

Folder creation failed or folder already exists: results/
Folder creation failed or folder already exists: results/log_g_store/
['agent_3', 'agent_1', 'agent_2', 'agent_0']
Box(-inf, inf, (40,), float32)
Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))


2025-12-21 08:34:30,393	WARNING services.py:2142 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.77gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-12-21 08:34:31,444	INFO worker.py:1927 -- Started a local Ray instance.


(raylet) It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="d31e04e4-3eec-4bb2-a9e8-2c475e23b505", ...)


### Policies

In [18]:
policies, policies_to_train = create_multi_agent_config(
    obs_space, act_space, num_agents, num_trained_agents=num_trained_agent)

policies: {'policy_0': <ray.rllib.policy.policy.PolicySpec object at 0x7578f89a7a90>, 'policy_1': <ray.rllib.policy.policy.PolicySpec object at 0x7578f89a7a30>, 'policy_2': <ray.rllib.policy.policy.PolicySpec object at 0x7578f89a7a60>, 'policy_3': <ray.rllib.policy.policy.PolicySpec object at 0x7578f89a7c40>}
policies_to_train: ['policy_0', 'policy_1']


### RLlib config

In [19]:
# Create callback instance with champion configuration
callback_instance = SelfPlayCallback(
    num_trainable_policies=num_trained_agent, 
    num_random_policies= num_agents - num_trained_agent,
    std_dev_multiplier=0.1,      # Snapshot when return > mean + 2*std
    max_champions=2,             # Keep last 5 champions (rolling window)
    min_iterations_between_champions=2,
)

In [20]:
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig

def get_config():

    config = (
        PPOConfig()
        .environment(
            "continuousDoubleAuction-v0",
            # continuousDoubleAuctionEnv,
            # env_config={
            #     "num_of_agents": num_of_traders,
            #     "init_cash": init_cash,
            #     "tick_size": tick_size,
            #     "tape_display_length": tape_display_length,
            #     "max_step": max_step - 1,
            #     "is_render": is_render,
            # }
            env_config=env_config,
            # env_config={"disable_env_checker": True},
        )
        .multi_agent(
            policies=policies,
            
            # policy_mapping_fn=policy_mapping_fn,
            policy_mapping_fn=SelfPlayCallback.get_mapping_fn(callback_instance),
            
            policies_to_train=policies_to_train,

            count_steps_by = "env_steps"  # DEFAULT - but this changes everything!
            # count_steps_by="agent_steps",  # ← ADD THIS!
        )
        # .training(
        #     model={
        #         "custom_model": CustomLSTMRLModule,
        #         # "custom_model_config": {
        #         #     "fcnet_hiddens": [256, 256],  # Neural network architecture
        #         #     "fcnet_activation": "relu",
        #         # },
        #     }
        # )
        .env_runners(
            # num_env_runners=num_workers,

            num_env_runners=0, 
            
            # num_envs_per_env_runner=num_envs_per_worker,
            # rollout_fragment_length=rollout_fragment_length,
            # batch_mode=batch_mode,
        )
        .learners(
            
            # Local Learner running on the main process (driver/head node).
            # Training runs on CPUs by default, or on a single GPU if num_gpus_per_learner > 0 is set. 
            # This is suitable for single-node training or simple, non-distributed setups.
            num_learners=0,  # Typically 1 learner unless using distributed training

            num_gpus_per_learner=num_gpus,  # Trainer GPU allocation
            # num_cpus_per_learner=num_cpus_per_worker,
        )
        .training(
            # train_batch_size_per_learner=train_batch_size / 4,
            train_batch_size_per_learner=train_batch_size,
            train_batch_size=train_batch_size,
            num_epochs=4,
        )
        # .callbacks(SelfPlayCallback)
        # .callbacks(lambda: SelfPlayCallback(win_rate_threshold=0.60))           
        # .callbacks(lambda: MinimalLeagueCallback(
        #     return_threshold=100.0,
        #     check_every_n_iters=1,
        # ))
        
        # .callbacks(lambda: SelfPlayCallback(
        #     # win_rate_threshold=0.10,
        #     ))
        .callbacks(lambda: callback_instance)

        # .output_dir(log_dir)
        .framework("torch")  # Explicitly set framework if needed
        .debugging(log_level="DEBUG")
        # .api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)
    )

    # # Optional: Configure resources more granularly if needed
    # if num_gpus_per_worker > 0:
    #     config.env_runners(
    #         num_gpus_per_env_runner=num_gpus_per_worker
    #     )

    return config

### Train


In [ ]:
def go_train(config):
    # trainer = ppo.PPOTrainer(config=config, env="continuousDoubleAuction-v0")

    # In your notebook, add this right before config.build():
    print("=" * 80)  
    print(f"DEBUG: train_batch_size = {train_batch_size}")
    print(f"DEBUG: Expected episodes per iter = {num_episodes_per_iter}")
    # print(f"DEBUG: Agent timesteps per episode = {agent_time_step_per_episode}")
    print("=" * 80)

    algo = config.build()

    print("=" * 80)
    print(f"ACTUAL CONFIG train_batch_size: {algo.config.train_batch_size}")
    print(f"ACTUAL CONFIG num_env_runners: {algo.config.num_env_runners}")
    print(f"ACTUAL CONFIG num_envs_per_env_runner: {algo.config.num_envs_per_env_runner}")  # ← KEY!
    print("=" * 80)

    # if is_restore == True:
    #     trainer.restore(restore_path)

    # g_store = ray.util.get_actor("g_store")
    # result = None
    for i in range(num_iters):
        result = algo.train()

    #     print(pretty_print(result)) # includes result["custom_metrics"]
    #     print("training loop = {} of {}".format(i + 1, num_iters))
    #     print("eps sampled so far {}".format(ray.get(g_store.get_eps_counter.remote())))

    #     if i % chkpt_freq == 0:
    #         checkpoint = algo.save(local_dir)
    #         print("checkpoint saved at", checkpoint)

    # checkpoint = algo.save(local_dir)
    # print("checkpoint saved at", checkpoint)
    # print("result['experiment_id']", result["experiment_id"])

                # Print step counts
        env_runner_results = result.get('env_runners', {})
        
        # print(f"\n=== Iteration {i+1} ===")
        # print(f"num_env_steps_sampled: {env_runner_results.get('num_env_steps_sampled', 'N/A')}")
        # print(f"num_agent_steps_sampled: {env_runner_results.get('num_agent_steps_sampled', 'N/A')}")
        # print(f"num_env_steps_trained: {env_runner_results.get('num_env_steps_trained', 'N/A')}")
        # print(f"num_agent_steps_trained: {env_runner_results.get('num_agent_steps_trained', 'N/A')}")

    # return result["experiment_id"]
    return None

# run everything
experiment_id = go_train(get_config())

2025-12-21 08:34:31,792	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-12-21 08:34:31,794	WARNING algorithm_config.py:5033 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


DEBUG: train_batch_size = 4096
DEBUG: Expected episodes per iter = 4


/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppre

ACTUAL CONFIG train_batch_size: 4096
ACTUAL CONFIG num_env_runners: 0
ACTUAL CONFIG num_envs_per_env_runner: 1

Candidates: ['policy_2', 'policy_3']

Candidates: ['policy_2', 'policy_3']

Episode b18f4caf93f94403af8acfb186073a92 Started - Policy Map:
  agent_0 -> policy_0
  agent_1 -> policy_1
  agent_2 -> policy_3
  agent_3 -> policy_2

on_episode_end:MAEps(len=1025 done=True Rs={'agent_0': -5228482252.147281, 'agent_1': -11565231983.441162, 'agent_2': -21688955507.15685, 'agent_3': -10363812597.400883} id_=b18f4caf93f94403af8acfb186073a92)

Candidates: ['policy_2', 'policy_3']

Candidates: ['policy_2', 'policy_3']

Episode 6427ea322e90432eac12df451bb4ba3e Started - Policy Map:
  agent_0 -> policy_0
  agent_1 -> policy_1
  agent_2 -> policy_2
  agent_3 -> policy_3

on_episode_end:MAEps(len=1025 done=True Rs={'agent_0': -13373108907.19605, 'agent_1': -7790937597.866859, 'agent_2': -15498570025.121262, 'agent_3': -9677247105.203918} id_=6427ea322e90432eac12df451bb4ba3e)

Candidates: ['p

In [ ]:
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_datetime)